<a href="https://colab.research.google.com/github/odlorah/haroldo_viana/blob/main/Music_App_project_by_HAROLDO_VIANA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv

def login():
  artistas = {} # Dicionário teporário para armazenar os artistas que serão adicionados ao aquivo. 
  # Criei ele aqui para nao ter o perigo de resetar caso o usuário volte ao menu inicial
  while True: # Início do programa. Identificação do usuário
    print("Bem-vindo ao sistema!")
    print("Entre com suas credenciais ou digite 'sair' para sair")
    username = input("Usuário: ")
    password = input("Senha: ")
        
    if username == "admin" and password == "admin123":
      print("Olá, admin!")              
      admin_login(artistas) # Abre função com funções do admin
      break
            
    elif username == "cliente" and password == "cliente123":
      print("Olá, cliente!")
      cliente_login(artistas) # Abre função com funções do cliente
      break
            
    elif username.lower() == "sair":
      print("Até logo!")
      break # Fecha o programa
            
    else:
      print("Credenciais inválidas. Tente novamente.")

def ler_informacoes(artistas):
  artistas_existentes = []
  albuns_existentes = []
  dados_completos = {}
  try:
    with open("banco_de_musicas.csv", "r") as file:
      for line in file:
        nome_artista, nome_album, musicas = line.strip().split(", ")
        if nome_artista in artistas:
          dados_completos[nome_artista][nome_album] = musicas.split(", ")
        else:
          dados_completos[nome_artista] = {nome_album: musicas.split(", ")}
        artistas_existentes.append(nome_artista)
        albuns_existentes.append(nome_album)
        if nome_album not in artistas[nome_artista]:
          dados_completos[nome_artista][nome_album] = []
  except FileNotFoundError:
    print("Arquivo de playlists não encontrado.")
  except ValueError:
    print("Arquivo de playlists vazio ou com informações inválidas.")

  return dados_completos, artistas_existentes, albuns_existentes


def admin_login(artistas):
  artistas_existentes, albuns_existentes = ler_informacoes(artistas)
  while True:
    escolha = input("Digite '1' para adicionar um artista ou '2' para adicionar um álbum. Ou digite 'sair' para sair: ")

    if escolha == '1':
      admin_artista(artistas_existentes, artistas)
      continue
    elif escolha == '2':
      admin_album(artistas_existentes, albuns_existentes, artistas)
      continue
    elif escolha.lower() == 'sair':
      print("Voltando ao menu principal.")
      break
    else:
      print("Opção inválida. Tente novamente.")

def admin_artista(artistas_existentes, artistas):
  while True:
    nome_artista = input("Digite o nome do artista (ou 'sair' para sair): ")

    if nome_artista.lower() == 'sair':
      break

    if nome_artista in artistas:
      print("Artista já cadastrado.")
      continuar = input("Deseja adicionar um álbum para esse artista? (s/n) ")
      if continuar.lower() == 's':
        admin_album(artistas_existentes, list(artistas[nome_artista]), artistas)
      continue

    if nome_artista in artistas_existentes:
      print("Artista já cadastrado no banco de playlists.")
      continue

    artistas[nome_artista] = {'' : []}
    print(f"Artista '{nome_artista}' adicionado com sucesso!")
    continuar = input("Deseja adicionar um álbum para esse artista? (s/n) ")
    if continuar.lower() == 's':
      admin_album(artistas_existentes, list(artistas[nome_artista]), artistas)
      continue

def admin_album(artistas_existentes, albuns_existentes, artistas):
  # Mostra os artistas já cadastrados
  print("Artistas cadastrados:")
  for artista in artistas_existentes:
    print("-", artista)
  for artista in artistas:
    print("-", artista)

  # Pede o nome do artista
  nome_artista = input("Digite o nome do artista: ")

  # Verifica se o artista já está cadastrado
  if nome_artista not in artistas_existentes and nome_artista not in artistas:
    print("Artista não cadastrado.")
    return

  # Pede o nome do álbum
  nome_album = input("Digite o nome do álbum: ")

  # Verifica se o álbum já está cadastrado
  if nome_album in artistas_existentes.get(nome_artista, []) or nome_album in albuns_existentes:
    print("Álbum já cadastrado.")
    deletar = input("Deseja deletar e recadastrar as músicas? (s/n): ")
    if deletar.lower() == "s":
      if nome_album in artistas_existentes.get(nome_artista, []):
        del artistas_existentes[nome_artista][nome_album]
      else:
        albuns_existentes.remove(nome_album)
    else:
      return

  # Pede a quantidade de músicas do álbum
  num_musicas = int(input("Digite a quantidade de músicas do álbum: "))

  # Pede o nome de cada música e adiciona à lista de músicas
  musicas = []
  for i in range(num_musicas):
    musica = input(f"Digite o nome da música {i+1}: ")
    musicas.append(musica)

  # Adiciona o álbum ao dicionário do artista ou à lista de álbuns existentes
  if nome_artista in artistas_existentes:
    artistas_existentes[nome_artista][nome_album] = musicas
  else:
    albuns_existentes.append(nome_album)
    artistas.setdefault(nome_artista, {})
    artistas[nome_artista][nome_album] = musicas

  # Escreve o dicionário no arquivo 
  with open("arquivo.txt", "w") as file:
    [file.write(f"{artista}, {album}, {', '.join(musicas)}\n") for artista, albuns in artistas.items() for album, musicas in albuns.items()]
    print(f"Álbum '{nome_album}' adicionado com sucesso!")

def cliente_login(artistas):
  # Dicionário com as informações das playlists que serão incluídas posteriormente
  playlists = {}

  while True:
    # Menu principal do cliente
    print("=== Menu Cliente ===\n")
    print("1. Buscar playlist\n")
    print("2. Criar playlist\n")
    print("3. Sair\n")
    opcao_cliente = int(input("Escolha uma opção: "))
    #Buscar playlist 
    if opcao_cliente == 1:
      opcao = input("Buscar por (P)laylist, (A)rtista ou (M)úsica? ").lower()
      if opcao == "p":
        nome_playlist = input("Digite o nome da playlist: ").lower()
        playlists_encontradas = []

        with open("playlist.csv", "r") as file:
          reader = csv.reader(file)
          for linha in reader:
            if linha[0].lower() == nome_playlist:
              playlists_encontradas.append(linha[1:])

        if len(playlists_encontradas) > 0:
          print(f"Playlists encontradas com o nome '{nome_playlist}':")
          for playlist in playlists_encontradas:
            print(f"- {playlist}")
        else:
          print(f"Nenhuma playlist encontrada com o nome '{nome_playlist}'.")

      elif opcao == "a":
        nome_artista = input("Digite o nome do artista: ").lower()
        playlists_encontradas = []

        with open("playlist.csv", "r") as file:
          reader = csv.reader(file)
          for linha in reader:
            if nome_artista in linha[1].lower() or nome_artista in linha[2].lower():
              playlists_encontradas.append(linha[0])

        if len(playlists_encontradas) > 0:
          playlists_encontradas = list(set(playlists_encontradas))
          print(f"Playlists encontradas com o artista '{nome_artista}':")
          for playlist in playlists_encontradas:
            print(f"- {playlist}")
        else:
          print(f"Nenhuma playlist encontrada com o artista '{nome_artista}'.")

      elif opcao == "m":
        nome_musica = input("Digite o nome da música: ").lower()
        playlists_encontradas = []

        with open("playlist.csv", "r") as file:
          reader = csv.reader(file)
          for linha in reader:
            if nome_musica in linha[2].lower():
              playlists_encontradas.append(linha[0])

        if len(playlists_encontradas) > 0:
          playlists_encontradas = list(set(playlists_encontradas))
          print(f"Playlists encontradas com a música '{nome_musica}':")
          for playlist in playlists_encontradas:
            print(f"- {playlist}")
        else:
          print(f"Nenhuma playlist encontrada com a música '{nome_musica}'.")
            
      else:
        print("Opção inválida.")
    # Cadastrar playlist
    elif opcao_cliente == 2:
      dados_completos, *resto = ler_informacoes(artistas)
      nome_playlist = input("Digite o nome da playlist: ")
      playlist = {}

      continuar_adicionando = True
      while continuar_adicionando:
        nome_do_artista = input("Digite o nome do artista: ")
        nome_do_album = input("Digite o nome do álbum: ")
        nome_da_musica = input("Digite o nome da música: ")
        # Verificando as entradas do usuário com as informações do arquivo do banco de dados
        if nome_do_artista in artistas and nome_do_album in artistas[nome_do_artista]:
          if nome_da_musica in artistas[nome_do_artista][nome_do_album]:
            if nome_playlist in playlist:
              playlist[nome_playlist].append([nome_do_artista, nome_do_album, nome_da_musica])
            else:
              playlist[nome_playlist] = [[nome_do_artista, nome_do_album, nome_da_musica]]
              print(f"Música '{nome_da_musica}' adicionada à playlist '{nome_playlist}'.")
          else:
            print(f"Música '{nome_da_musica}' não encontrada no álbum '{nome_do_album}'.")
        else:
          print(f"Álbum '{nome_do_album}' ou artista '{nome_do_artista}' não encontrado(s).")

      with open("playlist.csv", "a") as file:
        writer = csv.writer(file)
        for nome_playlist, musicas in playlist.items():
          for musica in musicas:
            writer.writerow([nome_playlist] + musica)

        print("Playlist salva com sucesso!")
    else:
      break

Observações:
1. Minha intenção seria ainda de criar um 'main' padrão do 'mundo Python, mas gostaria de tirar essa dúvida posteriormente de como realizar essa operação, então, até o momento, o 'main' do código é a função login()
2. Gostaria de aplicar o 'comprehension' em algumas das expressões que utilizam do 'for', mas devido a falta de tempo e a incerteza do prazo mínimo de entrega, preferi não arriscar
3. O uso das expressões do conceito de Programação Funcional também não foram aplicadas devido a falta de tempo restante para a produção do projeto, o 'for' ainda é o conceito de mais simples aplicação e ele foi preferido para que o projeto estivesse o mais possível completo
4. A demanda de criar um arquivo para a informação de álbuns não foi realizado devido a dúvida quanto a como relacionar os artistas à cada álbum, dado que eles estariam em arquivos diferentes
5. Não deixei o código como eu gostaria, mas um feedback sobre prazo foi compartilhado com a equipe e, infelizmente, "é isso o que temos para hoje"